In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 67kB/s 
     |████████████████████████████████| 204kB 46.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=5cc7f67d687e285f65c26c3f518b85d3171171f69da7d910c2aae32ae159b97c
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
import pyspark as ps
from pyspark.sql import SQLContext
from pyspark.ml.evaluation import RegressionEvaluator##Error evaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder## Hyper parameters
from pyspark.ml import Pipeline
from pyspark.sql import Row
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import udf, col, when
import numpy as np

In [ ]:
appName = "Recommender system in Spark"
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

Read file into dataFrame

In [ ]:
df=spark.read.json("/content/Automotive_5.json") 

In [ ]:
df.show(50)  # to display the data in the dataframe.

+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|B00002243X|  [4, 4]|    5.0|I needed a set of...|08 17, 2011|A3F73SC1LY51OO|     Alan Montgomery|Work Well - Shoul...|    1313539200|
|B00002243X|  [1, 1]|    4.0|These long cables...| 09 4, 2011|A20S66SKYXULG2|            alphonse|    Okay long cables|    1315094400|
|B00002243X|  [0, 0]|    5.0|Can't comment muc...|07 25, 2013|A2I8LFSN2IS5EO|               Chris|Looks and feels h...|    1374710400|
|B00002243X|[19, 19]|    5.0|I absolutley love...|12 21, 2010|A3GT2EWQSO45ZG|              DeusEx|Excellent choice ...|    1292889600|
|B00002243X|  [0, 0]|    5.0|I purchased the 1...| 07 4

In [ ]:
df.filter(col("reviewerID")=='A3EN2AV22EWE43').show() # filter is used to filter the rows based on a given condition. 

+----------+-------+-------+--------------------+-----------+--------------+------------+--------------------+--------------+
|      asin|helpful|overall|          reviewText| reviewTime|    reviewerID|reviewerName|             summary|unixReviewTime|
+----------+-------+-------+--------------------+-----------+--------------+------------+--------------------+--------------+
|B001EAWNVY| [0, 0]|    5.0|This product is w...|07 23, 2010|A3EN2AV22EWE43|     Tony W.|Amazing!!! Can no...|    1279843200|
|B003CG9PIC| [0, 1]|    4.0|I did lots of loo...|01 29, 2014|A3EN2AV22EWE43|     Tony W.|Heavy Duty Snatch...|    1390953600|
|B004BWU8H8| [0, 0]|    4.0|Install was super...|01 29, 2014|A3EN2AV22EWE43|     Tony W.|Excellent Locking...|    1390953600|
|B004ZF1MWG| [0, 0]|    4.0|Got these for my ...| 06 3, 2014|A3EN2AV22EWE43|     Tony W.|Great headlight u...|    1401753600|
|B00554PNL2| [0, 0]|    3.0|I bought this at ...|01 29, 2014|A3EN2AV22EWE43|     Tony W.| Good wiring harness|    1390

In [ ]:
df.filter(col("reviewerID")=='A3155NWLKXEY1I').show()

+----------+--------+-------+--------------------+-----------+--------------+--------------+--------------------+--------------+
|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|  reviewerName|             summary|unixReviewTime|
+----------+--------+-------+--------------------+-----------+--------------+--------------+--------------------+--------------+
|B00009RAX7|  [2, 2]|    5.0|I had a cracked a...|03 29, 2008|A3155NWLKXEY1I|AKO California|  Pretty easy to use|    1206748800|
|B0007QGT34|[12, 13]|    5.0|I didn't have any...|03 16, 2012|A3155NWLKXEY1I|AKO California|Car running very ...|    1331856000|
|B000CB7DG6|  [1, 1]|    5.0|This fits my 1998...| 04 4, 2014|A3155NWLKXEY1I|AKO California|Look up the part ...|    1396569600|
|B000UK3RT6|  [0, 1]|    4.0|I haven't ridden ...| 04 2, 2012|A3155NWLKXEY1I|AKO California|Will take time to...|    1333324800|
|B001DKTG0G|  [0, 0]|    5.0|I replaced the ma...|03 17, 2012|A3155NWLKXEY1I|AKO California|Car r

In [ ]:
# drop not needed columns
df1 = df.drop('helpful','reviewText','reviewTime','reviewerName','unixReviewTime','summary')

# rename columns, and placed all the values in a new dataframe.
df2 = df.selectExpr("asin","overall as rating", "reviewerID as user")
# show first 6 rows
df2.show(n=6)     
# display data types and check if they are correct, e.g. rating should be double
df2.printSchema()

+----------+------+--------------+
|      asin|rating|          user|
+----------+------+--------------+
|B00002243X|   5.0|A3F73SC1LY51OO|
|B00002243X|   4.0|A20S66SKYXULG2|
|B00002243X|   5.0|A2I8LFSN2IS5EO|
|B00002243X|   5.0|A3GT2EWQSO45ZG|
|B00002243X|   5.0|A3ESWJPAVRPWB4|
|B00002243X|   5.0| A1ORODEBRN64C|
+----------+------+--------------+
only showing top 6 rows

root
 |-- asin: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- user: string (nullable = true)



In [ ]:
df2.describe().show() # describe shows the statistical results.

+-------+----------+------------------+--------------------+
|summary|      asin|            rating|                user|
+-------+----------+------------------+--------------------+
|  count|     20473|             20473|               20473|
|   mean|      null| 4.471840961266058|                null|
| stddev|      null|0.9403365986838125|                null|
|    min|B00002243X|               1.0|A00473363TJ8YSZ3Y...|
|    max|B00KIAQ8VW|               5.0|       AZYBZEVISD0YX|
+-------+----------+------------------+--------------------+



In [ ]:
# transform asin and user alphanumeric string to index using spark StringIndexer function
from pyspark.ml.feature import StringIndexer 
asinIndexer = StringIndexer(inputCol="asin", outputCol="itemiD",handleInvalid='error') 
userIndexer = StringIndexer(inputCol='user',outputCol='userid',handleInvalid='error') 
asinIndexed = asinIndexer.fit(df2).transform(df2)  
df_indexed  = userIndexer.fit(asinIndexed).transform(asinIndexed)  

In [ ]:
df_indexed.show()


+----------+------+--------------+------+------+
|      asin|rating|          user|itemiD|userid|
+----------+------+--------------+------+------+
|B00002243X|   5.0|A3F73SC1LY51OO| 547.0|2499.0|
|B00002243X|   4.0|A20S66SKYXULG2| 547.0| 563.0|
|B00002243X|   5.0|A2I8LFSN2IS5EO| 547.0|  33.0|
|B00002243X|   5.0|A3GT2EWQSO45ZG| 547.0|  37.0|
|B00002243X|   5.0|A3ESWJPAVRPWB4| 547.0| 649.0|
|B00002243X|   5.0| A1ORODEBRN64C| 547.0| 219.0|
|B00002243X|   5.0|A2R49ZN3G6FTCQ| 547.0|1391.0|
|B00002243X|   5.0|A1Q65KYDKXIX8E| 547.0| 125.0|
|B00002243X|   4.0|A3BI8BKIHESDNQ| 547.0|2469.0|
|B00002243X|   5.0|A1R089P5AS26UE| 547.0|1972.0|
|B00002243Z|   4.0|A2TQST4IPVA0EC|1113.0|1403.0|
|B00002243Z|   5.0|A3MQAQT8C6D1I7|1113.0|1545.0|
|B00002243Z|   5.0|A1C6RZG7XQZBUZ|1113.0|  42.0|
|B00002243Z|   4.0|A3GT2EWQSO45ZG|1113.0|  37.0|
|B00002243Z|   4.0|A3I23XW4L5ITOG|1113.0| 191.0|
|B00002243Z|   4.0|A30ASPRK2VP8QC|1113.0| 318.0|
|B00008BKX5|   3.0|A1KKOA3FP2DMBW| 915.0|1912.0|
|B00008BKX5|   4.0|A

In [ ]:
training_df, validation_df = df_indexed.randomSplit([.8, .2])

In [ ]:
print("The total number of reviewers of the training data : ",training_df.select("user").count())

The total number of reviewers of the training data :  16315


In [ ]:
print("The total number of products of the training data : ",training_df.select("asin").count())

The total number of products of the training data :  16315


In [ ]:
iterations = 10  # hyper parameters
regularization_parameter = 0.1
rank= 4
errors = []
err = 0

In [ ]:
als = ALS(maxIter=iterations, regParam=regularization_parameter, rank=4, userCol="userid", itemCol="itemiD", ratingCol="rating")
model = als.fit(training_df)
predictions = model.transform(validation_df)
new_predictions = predictions.filter(col('prediction') != np.nan)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(new_predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 5.550375269370397


In [ ]:
predictions.show()


+----------+------+--------------+------+------+-----------+
|      asin|rating|          user|itemiD|userid| prediction|
+----------+------+--------------+------+------+-----------+
|B003CG9PIC|   5.0|A2IKO9YTH7QLWH| 148.0|2209.0|   2.915052|
|B003CG9PIC|   5.0|A2QR9IXLMIDL5U| 148.0|  40.0|  7.1163816|
|B003CG9PIC|   3.0|A17WU4OZ17UG0J| 148.0|1813.0| -1.3192189|
|B002M589TA|   5.0|A2CYY89WPXX68H| 463.0|2156.0|  2.8013816|
|B002M589TA|   5.0|A2VQBRCE1AMTN3| 463.0| 910.0|  2.9409618|
|B007OSL3DY|   5.0| AD0VA39FRVNKN| 471.0| 348.0|  -4.406698|
|B007OSL3DY|   4.0|A2CDGZWYTUTM2E| 471.0|2151.0| 0.05765581|
|B00144I5KS|   2.0|A3EGHUQPUV6E5U| 833.0|2493.0|    4.30387|
|B0067ECRP4|   5.0|A185O1MX1RP964|1088.0|  52.0|  7.0616765|
|B0067ECRP4|   5.0|A3I42CDG116KKQ|1088.0|  41.0|  2.0123353|
|B0067ECRP4|   5.0|A27FBC56WXTH0J|1088.0|2119.0| 0.28720045|
|B001AWLTX4|   5.0|A3UXW18DP4WSD6|1238.0| 196.0|  -4.378708|
|B001AWLTX4|   5.0|A3HF8SHOKQ25YS|1238.0|2512.0| -2.6383488|
|B001AWLTX4|   4.0|A3GU1

In [ ]:
final_als = ALS(maxIter=10, regParam=0.1, rank=13, userCol="userid", itemCol="itemiD", ratingCol="rating")
final_model = final_als.fit(training_df)
predictions = final_model.transform(validation_df)
new_predictions = predictions.filter(col('prediction') != np.nan)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(new_predictions)
print("Root-mean-square error of the ALS model = " + str(rmse))

Root-mean-square error of the ALS model = 1.3273162922448143


In [ ]:
predictions = final_model.transform(validation_df)
predictions.show(n = 10)                      

+----------+------+--------------+------+------+----------+
|      asin|rating|          user|itemiD|userid|prediction|
+----------+------+--------------+------+------+----------+
|B003CG9PIC|   5.0|A2IKO9YTH7QLWH| 148.0|2209.0|  4.547538|
|B003CG9PIC|   5.0|A2QR9IXLMIDL5U| 148.0|  40.0| 3.1852183|
|B003CG9PIC|   3.0|A17WU4OZ17UG0J| 148.0|1813.0| 2.9878469|
|B002M589TA|   5.0|A2CYY89WPXX68H| 463.0|2156.0| 4.2622137|
|B002M589TA|   5.0|A2VQBRCE1AMTN3| 463.0| 910.0| 2.9270933|
|B007OSL3DY|   5.0| AD0VA39FRVNKN| 471.0| 348.0|  3.334535|
|B007OSL3DY|   4.0|A2CDGZWYTUTM2E| 471.0|2151.0| 3.9260328|
|B00144I5KS|   2.0|A3EGHUQPUV6E5U| 833.0|2493.0| 4.6990914|
|B0067ECRP4|   5.0|A185O1MX1RP964|1088.0|  52.0| 3.8168762|
|B0067ECRP4|   5.0|A3I42CDG116KKQ|1088.0|  41.0|  4.634039|
+----------+------+--------------+------+------+----------+
only showing top 10 rows



In [ ]:
users1 = df_indexed .select(final_als.getUserCol()).distinct().limit(3)
userSubsetRecs = final_model.recommendForUserSubset(users1, 10)


In [ ]:
userSubsetRecs.show(10,False)

+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userid|recommendations                                                                                                                                                                       |
+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1051  |[{1670, 5.332966}, {1233, 5.1821856}, {1169, 5.1526575}, {1497, 5.1271563}, {610, 5.1134367}, {1800, 5.054937}, {597, 5.053345}, {1323, 5.020869}, {947, 5.0101147}, {607, 4.9807463}]|
|596   |[{1662, 5.009604}, {998, 5.004603}, {372, 4.9458838}, {1753, 4.938416}, {903, 4.9215617}, {1101, 4.8614693}, {573, 4.837417}, {1169, 4.8349233}, {1742, 4.798128}, {842, 4.7899933}]  |
|299   |[{451, 5.8972397}, {715, 5.87917

In [ ]:
user = df_indexed.select('user',"userid").distinct().filter(col("user")=='A3EN2AV22EWE43')

In [ ]:
user.show()

+--------------+------+
|          user|userid|
+--------------+------+
|A3EN2AV22EWE43|2496.0|
+--------------+------+



In [ ]:
userRecs = final_model.recommendForUserSubset(user.select("userid"), 10)

In [ ]:
userRecs.show(5,False)

+------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userid|recommendations                                                                                                                                                                        |
+------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|2496  |[{1535, 4.835552}, {1268, 4.8165054}, {1142, 4.815558}, {1080, 4.814888}, {1249, 4.750426}, {1577, 4.6716714}, {1204, 4.649025}, {1585, 4.6378865}, {1371, 4.588948}, {871, 4.5568895}]|
+------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+



In [ ]:
import pandas as pd
recs = userRecs.select("recommendations").collect()[0][0]
pdf = pd.DataFrame(recs, columns = ['itemiD','rating1'])
print(pdf)


   itemiD   rating1
0    1535  4.835552
1    1268  4.816505
2    1142  4.815558
3    1080  4.814888
4    1249  4.750426
5    1577  4.671671
6    1204  4.649025
7    1585  4.637887
8    1371  4.588948
9     871  4.556890


In [ ]:
sparkDF2 = spark.createDataFrame(pdf)

In [ ]:
df3=sparkDF2.join(df_indexed,"itemId")

The 10 products recommended for the user (reviewer): A3EN2AV22EWE43 & the predicted ratings of the 10 products.


In [ ]:
print("The 10 products recommended for the user (reviewer): A3EN2AV22EWE43 ")

df3.select("itemiD","rating1","asin").distinct().show()

The 10 products recommended for the user (reviewer): A3EN2AV22EWE43 
+------+------------------+----------+
|itemiD|           rating1|      asin|
+------+------------------+----------+
|  1585|4.6378865242004395|B000SQFD4O|
|  1142| 4.815557956695557|B0007O8S58|
|  1535| 4.835552215576172|B000FQ7L8I|
|  1577| 4.671671390533447|B000PWZR4C|
|  1080| 4.814888000488281|B005JU5UEW|
|  1371| 4.588947772979736|B009D3E36W|
|  1268| 4.816505432128906|B001VZL2LO|
|   871| 4.556889533996582|B003IOITRG|
|  1204| 4.649024963378906|B000I14RY6|
|  1249| 4.750425815582275|B001EAWNVY|
+------+------------------+----------+

